In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('17_cities.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,Country,Country,3,CREATE TABLE Country\n(countryid INTEGER PRIMA...
1,table,State,State,2,CREATE TABLE State\n(stateid INTEGER PRIMARY K...


In [6]:
try:
    cursor = conn.cursor()
    cursor.execute('''
    DROP TABLE Country;
    ''')
    cursor.close()
except Exception as e:
    print(e)

In [7]:
try:
    cursor = conn.cursor()
    cursor.execute('''
    DROP TABLE State;
    ''')
    cursor.close()
except Exception as e:
    print(e)

In [8]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE Country
(countryid INTEGER PRIMARY KEY,
countryName TEXT);
''')
cursor.close()

In [9]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE State
(stateid INTEGER PRIMARY KEY,
countryid INTEGER,
stateName TEXT);
''')
cursor.close()

In [10]:
cursor = conn.cursor()
cursor.execute('''INSERT INTO Country VALUES (NULL, 'India');''')
cursor.execute('''INSERT INTO Country VALUES (NULL, 'Nepal');''')
cursor.execute('''INSERT INTO Country VALUES (NULL, 'United States');''')
cursor.execute('''INSERT INTO Country VALUES (NULL, 'Canada');''')
cursor.execute('''INSERT INTO Country VALUES (NULL, 'Sri Lanka');''')
cursor.execute('''INSERT INTO Country VALUES (NULL, 'Brazil');''')
cursor.close()

In [11]:
pd.read_sql_query(sql='''
SELECT *
FROM Country;
''', con=conn)

,countryid,countryName
0,1,India
1,2,Nepal
2,3,United States
3,4,Canada
4,5,Sri Lanka
5,6,Brazil


In [12]:
cursor = conn.cursor()
cursor.execute('''INSERT INTO State VALUES (NULL, 1, 'Maharashtra');''')
cursor.execute('''INSERT INTO State VALUES (NULL, 1, 'Punjab');''')
cursor.execute('''INSERT INTO State VALUES (NULL, 2, 'Kathmandu');''')
cursor.execute('''INSERT INTO State VALUES (NULL, 3, 'California');''')
cursor.execute('''INSERT INTO State VALUES (NULL, 3, 'Texas');''')
cursor.execute('''INSERT INTO State VALUES (NULL, 4, 'Alberta');''')
cursor.close()

In [13]:
pd.read_sql_query(sql='''
SELECT *
FROM State;
''', con=conn)

,stateid,countryid,stateName
0,1,1,Maharashtra
1,2,1,Punjab
2,3,2,Kathmandu
3,4,3,California
4,5,3,Texas
5,6,4,Alberta


In [14]:
pd.read_sql_query(sql='''
SELECT c.countryid, c.countryName, s.stateName
FROM Country c
JOIN State s
ON c.countryid = s.countryid;
''', con=conn)

,countryid,countryName,stateName
0,1,India,Maharashtra
1,1,India,Punjab
2,2,Nepal,Kathmandu
3,3,United States,California
4,3,United States,Texas
5,4,Canada,Alberta


In [15]:
pd.read_sql_query(sql='''
SELECT c.countryid, c.countryName, s.stateName
FROM Country c
LEFT JOIN State s
ON c.countryid = s.countryid;
''', con=conn)

,countryid,countryName,stateName
0,1,India,Maharashtra
1,1,India,Punjab
2,2,Nepal,Kathmandu
3,3,United States,California
4,3,United States,Texas
5,4,Canada,Alberta
6,5,Sri Lanka,None
7,6,Brazil,None


In [16]:
pd.read_sql_query(sql='''
SELECT c.countryid, c.countryName, s.stateName
FROM State s
LEFT JOIN Country c
ON c.countryid = s.countryid;
''', con=conn)

,countryid,countryName,stateName
0,1,India,Maharashtra
1,1,India,Punjab
2,2,Nepal,Kathmandu
3,3,United States,California
4,3,United States,Texas
5,4,Canada,Alberta


# RIGHT and FULL OUTER JOINs are not currently supported

pd.read_sql_query(sql='''
SELECT c.countryid, c.countryName, s.stateName
FROM State s
OUTER JOIN Country c
ON c.countryid = s.countryid;
''', con=conn)

SELECT d.type,
         d.color,
         c.type,
         c.color
FROM dogs d
LEFT JOIN cats c USING(color)
UNION ALL
SELECT d.type,
         d.color,
         c.type,
         c.color
FROM cats c
LEFT JOIN dogs d USING(color)
WHERE d.color IS NULL;

In [17]:
cursor = conn.cursor()
cursor.execute('''INSERT INTO State VALUES (NULL, 7, 'Georgetown');''')
cursor.close()

In [18]:
pd.read_sql_query(sql='''
SELECT c.countryid, c.countryName, s.stateName
FROM Country c
LEFT JOIN State s
ON c.countryid = s.countryid;
''', con=conn)

,countryid,countryName,stateName
0,1,India,Maharashtra
1,1,India,Punjab
2,2,Nepal,Kathmandu
3,3,United States,California
4,3,United States,Texas
5,4,Canada,Alberta
6,5,Sri Lanka,None
7,6,Brazil,None


In [19]:
pd.read_sql_query(sql='''
SELECT c.countryid, c.countryName, s.stateName
FROM State s
LEFT JOIN Country c
ON c.countryid = s.countryid
WHERE c.countryid IS NULL;
''', con=conn)

,countryid,countryName,stateName
0,None,None,Georgetown


In [20]:
pd.read_sql_query(sql='''
SELECT c.countryid, c.countryName, s.stateName
FROM Country c
LEFT JOIN State s
ON c.countryid = s.countryid
UNION ALL
SELECT c.countryid, c.countryName, s.stateName
FROM State s
LEFT JOIN Country c
ON c.countryid = s.countryid
WHERE c.countryid IS NULL;
''', con=conn)

,countryid,countryName,stateName
0,1.0,India,Maharashtra
1,1.0,India,Punjab
2,2.0,Nepal,Kathmandu
3,3.0,United States,California
4,3.0,United States,Texas
5,4.0,Canada,Alberta
6,5.0,Sri Lanka,None
7,6.0,Brazil,None
8,NaN,None,Georgetown


# my test

inner join, more rows from Country, more rows from State

In [29]:
pd.read_sql_query(sql='''
SELECT c.countryid c_countryid, c.countryName, s.stateid, s.countryid s_countryid, s.stateName
FROM Country c
JOIN State s
ON c.countryid = s.countryid;
''', con=conn)

,c_countryid,countryName,stateid,s_countryid,stateName
0,1,India,1,1,Maharashtra
1,1,India,2,1,Punjab
2,2,Nepal,3,2,Kathmandu
3,3,United States,4,3,California
4,3,United States,5,3,Texas
5,4,Canada,6,4,Alberta


In [32]:
pd.read_sql_query(sql='''
SELECT c.countryid c_countryid, c.countryName, s.stateid, s.countryid s_countryid, s.stateName
FROM Country c
LEFT JOIN State s
ON c.countryid = s.countryid
WHERE s.countryid IS NULL;
''', con=conn)

,c_countryid,countryName,stateid,s_countryid,stateName
0,5,Sri Lanka,None,None,None
1,6,Brazil,None,None,None


In [33]:
pd.read_sql_query(sql='''
SELECT c.countryid c_countryid, c.countryName, s.stateid, s.countryid s_countryid, s.stateName
FROM State s
LEFT JOIN Country c
ON c.countryid = s.countryid
WHERE c.countryid IS NULL;
''', con=conn)

,c_countryid,countryName,stateid,s_countryid,stateName
0,None,None,7,7,Georgetown
